In [1]:
%load_ext autoreload
%autoreload 2
from IPython.core.debugger import set_trace

In [2]:
%matplotlib widget
import matplotlib.pyplot as plt

In [3]:
import pandas as pd
import numpy as np
from ast import literal_eval
from ecephys_analyses.data import paths, channel_groups
from ecephys.scoring import load_datetime_hypnogram
from ecephys.plot import plot_hypnogram_overlay
import seaborn as sns

In [4]:
def load_datetime_sr_chans(path):
    df = pd.read_csv(path, sep="\t", converters={"sr_chans": literal_eval})
    df["start_time"] = pd.to_datetime(df["start_time"])
    df["end_time"] = pd.to_datetime(df["end_time"])
    df["duration"] = pd.to_timedelta(df["duration"])

    return df

In [5]:
def load_condition_hypnogram(subject, experiment, condition):
    hypnogram_paths = paths.get_sglx_style_datapaths(subject, experiment, condition, ext="hypnogram.tsv")
    hypnograms = [load_datetime_hypnogram(path) for path in hypnogram_paths]
    return pd.concat(hypnograms).reset_index(drop=True)
    
def load_condition_sr_chans(subject, experiment, condition):
    sr_chans_paths = paths.get_sglx_style_datapaths(subject, experiment, condition, ext="sr_chans.tsv")
    sr_chans = [load_datetime_sr_chans(path) for path in sr_chans_paths]
    return pd.concat(sr_chans).reset_index(drop=True)

In [20]:
subject="Allan"
chan_map = channel_groups.full["Allan"]

In [21]:
df = load_condition_sr_chans(subject, "sleep-homeostasis", "all")

In [22]:
chan_depths = np.arange(len(chan_map)) * 20

In [23]:
df["sr_chan_depths"] = df.sr_chans.apply(lambda x: chan_depths[np.isin(chan_map, x)])
df["sr_depth"] = df.sr_chan_depths.apply(np.mean)

/home/gfindlay/miniconda3/envs/ecephys/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)


In [24]:
light = load_condition_hypnogram(subject, "sleep-homeostasis", "light-period")
light_cm = load_condition_hypnogram(subject, "sleep-homeostasis", "light-period-circadian-match")

In [28]:
fig, ax = plt.subplots(figsize=(22, 2))
ax.hlines(df['sr_depth'], df['start_time'], df['end_time'])
ax.set(ylabel="Distance from tip (um)", xlabel="Time")
plot_hypnogram_overlay(light, ax=ax)
plot_hypnogram_overlay(light_cm, ax=ax)
ax.set(xmargin=0)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[None]